In [ ]:
import sys,os 
sys.path.append('/home/benr/ACT/CW2/py')
import numpy as np 
import pandas as pd 
from PIL import Image, UnidentifiedImageError
from sklearn.preprocessing import LabelEncoder
from functions import get_data, galaxy_type,split_data_torch,train
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

: 

In [3]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

CUDA available: False
CUDA version: 12.8
CUDA_VISIBLE_DEVICES: None


# Question 2, Convolutional Neural Network for image recognition 
In this section the aim is to repeat the same process as question 1 but replace the traditional reandom forest method with a convolutional neural network. The expectation is that due to the CNN's ability to correlate spacial features, it will be a more roubust method of recognising galaxy morphology.

The first step will be to load the data in exactly the same way as Q1. 

In [2]:

# get the data from functions.py
images,labels = get_data()
# convert images to pytorch tnesors
images = torch.from_numpy(images)
labels = torch.from_numpy(labels).long()

print(images.shape)


/home/benr/.astroNN/datasets/Galaxy10_DECals.h5 was found!
torch.Size([17736, 256, 256, 3])


In [3]:
# Move channels: (N,H,W,C) -> (N,C,H,W)
images = images.permute(0, 3, 1, 2)      

# Normalise 
images = images.float() / 255.0      
print(images.shape)

torch.Size([17736, 3, 256, 256])


For the CNN the lables will need to be encoded into numerical values. 
Documentation can be found here: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

Now we set up the class that will hold the structure of the network. Galaxy_CNN takes the number of input channels, the number of classes (Prediction catagories) and passes them two the different layers of the network. 
Throughout the process of data being passed through different levels of the CNN we will need to keep trach of the network is change the size of each tensor. This can be done with the following formulas

* CNN Output - $$\frac{H + 2P -K}{s} + 1 $$
* Pooling Output - $$\frac{H-K}{s}  + 1$$

H = input height of width of image 


K = kernel size

P = padding size

s = stride 

In [4]:

class Galaxy_CNN(nn.Module):
    def __init__(self,inCH, nCL):  
        
        super(Galaxy_CNN,self).__init__()
        #first convolution layer
        self.conv1 = nn.Conv2d(3, inCH*2,kernel_size= 5,stride=1,padding =2)
        self.conv2 = nn.Conv2d(inCH*2,inCH*2, kernel_size=3,stride = 1,padding=1)
        self.conv3 = nn.Conv2d(inCH*2,inCH,kernel_size= 3,stride=1,padding=1)
        #linear output layer
        self.lin = nn.Linear(inCH * 32**2,nCL)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.15)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0),-1)
        x = self.lin(x)
        return x 

In [5]:
#split tensor into training and test sets (see functions.py)
I = np.arange(len(labels))
xtrn,xtst,ytrn,ytst = split_data_torch(images,labels,I)

train_ds = TensorDataset(xtrn, ytrn)
test_ds  = TensorDataset(xtst, ytst)

train_loader = DataLoader(train_ds, batch_size=12, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=12, shuffle=False)




In [10]:
num_classes = len(labels.unique())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Galaxy_CNN(64,num_classes).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=10**-3,weight_decay=0.0001)

print(torch.cuda.current_device())


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [7]:
train(model, train_loader, test_loader, criterion, optimizer, device, n_epoch=60)


Epoch 1/60 Train loss: 1.8962  Train acc: 0.296  Test acc: 0.414
Epoch 2/60 Train loss: 1.5177  Train acc: 0.459  Test acc: 0.443


KeyboardInterrupt: 